Read the various data files containing the overlap integrals, electronic kinetic energy integrals,
electron-nuclear attraction integrals, and the electron-electron repulsion integrals.

The overlap integrals are of the form
\begin{equation}
S_{\mu \nu} = \int \phi_{\mu}^*({\bf r}) \phi_{\nu}({\bf r}) d {\bf r},
\end{equation}
where $\phi_{\mu}$ and $\phi_{\nu}$ label distinct atomic orbitals centered on a given atom center, and 
${\bf r}$ is the electronic coordinates of the electron described by the atomic orbital.
The electronic kinetic energy integrals are of the form
\begin{equation}
T_{\mu \nu} = \int \phi_{\mu}^*({\bf r}) \left( -\frac{1}{2} \nabla_r^2 \right) \phi_{\nu}({\bf r}) d {\bf r},
\end{equation}
the electron-nuclear attraction integrals are of the form
\begin{equation}
V_{\mu \nu} = \int \phi_{\mu}^*({\bf r}) \left( -\sum_A^N \frac{Z_A}{r_A} \right) \phi_{\nu}({\bf r}) d {\bf r},
\end{equation}
and the electron-electron repulsion integrals are of the form
\begin{equation}
V^{\lambda \sigma}_{\mu \nu} = (\mu \nu | \lambda \sigma) = 
\int \phi_{\mu}^*({\bf r}_1) \phi_{\nu}({\bf r}_1)  -\frac{1}{r_{12}} \phi^*_{\lambda}({\bf r}_2) \phi_{\sigma}({\bf r}_2)d {\bf r}_1 d {\bf r}_2.
\end{equation}

In other words, the one-electron quantities (overlap, electron kinetic, and electron-nuclear attraction integrals) can be stored in arrays with 2 indices, and the two-electron quantities (electron-electron repulsion integrals) can be stored in an array with 4 indices.

The following functions will help sort the data from the various integral files into 2- and 4-index arrays, as appropriate.

In [ ]:
import numpy as np

### read the total nuclear repulsion energy, this is just a single number
nuc = np.loadtxt("enuc.dat")
### read the kinetic energy integrals
T    = np.loadtxt("T.dat")
### read the electronic-nuclear attraction integrals
V1   = np.loadtxt("V1.dat")
### read the electron-electron repulsion integrals
V2   = np.loadtxt("V2.dat")
### read the overlap integrals
S    = np.loadtxt("S.dat")

The integral files also contained their indices, so the arrays that were created by `np.loadtxt` are not
particularly convenient for manipulating the integral data for subsequent SCF (or correlated) calculations.
The following two functions will map the integral data to arrays that are ordered by the 
atomic orbital indices in precisely the way that is incidated in the equations above 
(except for the fact that the array indexing begins with zero in python).  In otherwords, we would like
to have an array `S_mat` that stores the overlap integrals such that element `S_mat[0,1]` corresponds
to the overlap integral $S_{1,2}$ in the equation above, an array `V2_mat` that stores the 2-electron
repulsion integrals such that element `V2_mat[0,1,2,3]` corresponds to the integral $V_{1,2}^{3,4}$ above, etc.


In [12]:
def map_1_e_array(dim, mat):
    ''' Pass the list that results from reading any
        of the 1-electron integral files to this function 
        and it will return a square matrix where the rows
        are ordered by the \mu index and the columns are 
        ordered by the \nu index.  Keep in mind that the data files for
        the 1-electron integrals store only the upper-triangle.
        dim is the number of basis functions'''
    idx = 0
    Mmat = np.zeros((dim,dim))
    for i in range(0,dim):
        for j in range(i,dim):
            ival = int(mat[idx,0]-1)
            jval = int(mat[idx,1]-1)
            Mmat[ival,jval] = mat[idx,2]
            Mmat[jval,ival] = mat[idx,2]
            idx = idx + 1
            
    return Mmat


def map_2_e_array(length, dim, mat):
    ''' Pass the list that results from reading the 
        2-electron repulsion integral files to this function 
        and it will return a 4-index square tensor ordered
        by \mu, \nu, \lambda, \sigma.  Keep in mind the data file
        only contains the non-zero integrals, and also only symmetry unique integrals 
        (there are 8 equivalent permutations of the indices, see the permutations below).
        length is the number of non-zero two electron elements 
        and dim is the number of basis functions. '''
    Mmat = np.zeros((dim,dim,dim,dim))
    for m in range(0, length):
        i = int(mat[m,0]-1)
        j = int(mat[m,1]-1)
        k = int(mat[m,2]-1)
        l = int(mat[m,3]-1)
        val = mat[m,4]
        Mmat[i,j,k,l] = val
        Mmat[j,i,k,l] = val
        Mmat[i,j,l,k] = val
        Mmat[j,i,l,k] = val
        Mmat[k,l,i,j] = val
        Mmat[l,k,i,j] = val
        Mmat[k,l,j,i] = val
        Mmat[l,k,j,i] = val
        
    return Mmat

We will call these functions in the cell below to map the initial arrays formed from reading
the integral files to arrays that are more amenable to manipulation.

In [17]:
### get the number of non-zero 2-electron integrals from length of V2 data set
v2_length = int(len(V2))
### get the number of basis functions from the maximum \mu index in the
### overlap data file
dim = int(S[len(S)-1][0])

### store overlap integrals in a matrix
S_mat = map_1_e_array(dim, S)
### store kinetic energy integrals in a matrix
T_mat = map_1_e_array(dim, T)
### store electron-nuclear attraction integrals in a matrix
V1_mat = map_1_e_array(dim, V1)
### store 2-electron integrals in a 4-index tensor
V2_mat = map_2_e_array(v2_length, dim, V2)